In [2]:
import requests
import re
from bs4 import BeautifulSoup
import math

https://www.scrapingdog.com/blog/scrape-linkedin-jobs/

# Target URL
Follow the steps to produce a target URL:
1. Goto Linkedin and search for a job with any filters you want.
2. Open up the Developer inspector (Use Google Chrome) and scroll down until the next set of jobs load.
3. In the network tab there should be a "search" request when it loads the new jobs.
4. Grab that URL in the search request.
5. replace the last NUMBER in the URL with a {} in order to utilize a .format() for looping later. ie 'start=0' -> 'start={}'

In [25]:
target_url = 'https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Data%2BScientist&location=United%2BStates&geoId=103644278&f_JT=F&f_E=2&f_WT=2&currentJobId=3664087381&position=1&pageNum=0&start={}'

In [26]:
target_url

'https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Data%2BScientist&location=United%2BStates&geoId=103644278&f_JT=F&f_E=2&f_WT=2&currentJobId=3664087381&position=1&pageNum=0&start={}'

# Job ID Loop

In [ ]:
l=[] #Empty list to store job IDs. These IDs are used to create the URLs for specific job searches later
a = 0 #Used for the URL page number
pattern = 'jobPosting:(?P<job_id>\d+)' #Regex patter to get the job ID
for i in range(0, math.ceil(7807/25)): #Use math.ceil() to round off the loop
    res = requests.get(target_url.format(a)) #Get request against the target URL and a .format() in order to provide specific page number.
    a += 25
    soup = BeautifulSoup(res.text, 'html.parser')
    all_jobs = soup.find_all('li') #The 'li' tag contains the job ID
    #print(len(all_jobs)) #Debug
    for j in range(0, len(all_jobs)): #Loop through all b4 'li' list
        job_id = re.search(pattern, str(all_jobs[j])) #Using Regex to search for the ID
        l.append(int(job_id.groups()[0])) #Appened the ID and casting it as int for later use.

# Job Loop

In [109]:
target_job_url = 'https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/{}/'

In [110]:
test_target_job_url = 'https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/3664087381/'

In [115]:
test_res_job =  requests.get(test_target_job_url)
test_soup = BeautifulSoup(test_res_job.text, 'html.parser')
test_soup.find("div",{"class":"top-card-layout__card"}).find("a").find("img").get('alt')

'Analytica'

In [121]:
test_soup.find("div",{"class":"top-card-layout__entity-info"}).find("a").text.strip()

'Data Scientist'

In [113]:
test_soup


<!-- -->
<section class="top-card-layout container-lined overflow-hidden babybear:rounded-[0px]">
<div class="top-card-layout__card relative p-2 papabear:p-details-container-padding">
<a data-tracking-control-name="public_jobs_topcard_logo" data-tracking-will-navigate="" href="https://www.linkedin.com/company/analytica-inc?trk=public_jobs_topcard_logo" target="_self">
<img alt="Analytica" class="artdeco-entity-image artdeco-entity-image--square-5" data-delayed-url="https://media.licdn.com/dms/image/C4E0BAQGyzLU9Qb95hw/company-logo_100_100/0/1658342420131?e=2147483647&amp;v=beta&amp;t=iCOXMIyPJUYOY0QELowZzhYBJ2mskqPyZl8a4ULSdPc" data-ghost-classes="artdeco-entity-image--ghost" data-ghost-url="https://static.licdn.com/aero-v1/sc/h/9a9u41thxt325ucfh5z8ga4m8"/>
</a>
<div class="top-card-layout__entity-info-container flex flex-wrap papabear:flex-nowrap">
<div class="top-card-layout__entity-info flex-grow flex-shrink-0 basis-0 babybear:flex-none babybear:w-full babybear:flex-none babybear:w

In [124]:
test_soup.find("div",{"class":"show-more-less-html__markup"}).find('p').text

'Analytica is seeking a remoteData Scientistto support one or more dynamic, long-term federal government data science programs. The company works as a trusted advisor to U.S. federal government clients in health, civilian, and national security missions.The ideal candidate will be comfortable as a key member of a multi-disciplinary team working in an Agile environment to produce advanced analytics solutions.Analytica has been recognized byInc. Magazineas one of the 250 fastest-growing US small businesses for three consecutive years.We work with U.S. government clients to build data-driven products and cultures that make an impact on our daily lives. Analytica offers competitive compensation with opportunities for bonuses, employer-paid health care, unlimited training funds, and a 401k match.Requirements:'

In [125]:
test_soup.find("div",{"class":"show-more-less-html__markup"}).find('ul').text

'Feature Engineering and Attribute Evaluation - The candidate must demonstrate experience with identifying the key determinants for modeling that exist in the business process and within existing data sets, as well as selecting evaluation protocols (model techniques).Pre-processing - Demonstrate the skills and experience to collect, clean, and prepare data sets for input into a computational model using technologies such as Python or R. Candidates will explain various methods you have applied using common pre-processing functions.Modeling - Candidates will have practiced skills and experience selecting and applying modeling techniques to fit the business problem. Ideal candidatesshould be able to applymachine learning (ML) supervised and unsupervised learning, regression, neural networks and deep learning, natural language processing, etc.Validation - Candidates will have experience investigating, testing, reporting, and justifying model results.Visualization – Candidates should be com